In [1]:
# Vol Surface Constructor

# 1/ Collect Data from YF
# 2/ Process and clean data
# 3/ Calculate IV
# 4/ Contsruct Vol Surface

In [2]:
# Updates

# Make the code more pythonic (use functions and call them rather than jsut blocks of code)
      # ---> made progress with this (almsot done)

# figure out why that one outlier isnt being removed

# maybe add logic where we can mnaually remove outliers

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

In [4]:
# INPUTS
ticker = 'SPY'
outlier_penalty = 1
k = 5   # no. of neighbours in KNN


In [5]:
# Import data and convert into dataframe
def import_data(ticker):
    stock_data = yf.Ticker(ticker)
    expiration_dates = stock_data.options
    option_data = []

    # Loop through all expiration dates and collect the data
    for exp in expiration_dates:
        option_chain = stock_data.option_chain(exp)
        
        calls = option_chain.calls
        puts = option_chain.puts
        
        calls['expiration_date'] = exp
        calls['type'] = 'call'
        puts['expiration_date'] = exp
        puts['type'] = 'put'
        
        option_data.append(calls)
        option_data.append(puts)

    options_df = pd.DataFrame(pd.concat(option_data, ignore_index=True))

    return options_df

options_df = import_data(ticker)
options_df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date,type
0,SPY250106C00370000,2024-12-31 20:39:02+00:00,370.0,216.76,221.03,222.27,0.000000,0.000000,695.0,398,2.228520,True,REGULAR,USD,2025-01-06,call
1,SPY250106C00500000,2025-01-03 17:03:42+00:00,500.0,90.71,91.05,92.28,-4.010002,-4.233533,2.0,1,0.907227,True,REGULAR,USD,2025-01-06,call
2,SPY250106C00505000,2024-12-27 20:43:47+00:00,505.0,88.42,86.05,87.28,0.000000,0.000000,1.0,1,0.861330,True,REGULAR,USD,2025-01-06,call
3,SPY250106C00530000,2024-12-31 19:05:28+00:00,530.0,55.75,61.06,62.29,0.000000,0.000000,480.0,490,0.635746,True,REGULAR,USD,2025-01-06,call
4,SPY250106C00550000,2025-01-03 20:34:43+00:00,550.0,42.06,41.07,42.30,1.090000,2.660484,862.0,17,0.454595,True,REGULAR,USD,2025-01-06,call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6820,SPY270115P00745000,2025-01-03 19:27:51+00:00,745.0,152.92,151.00,155.90,9.119995,6.342138,2.0,2,0.107644,True,REGULAR,USD,2027-01-15,put
6821,SPY270115P00750000,2024-12-24 17:18:36+00:00,750.0,150.02,156.00,160.90,0.000000,0.000000,NaN,0,0.109994,True,REGULAR,USD,2027-01-15,put
6822,SPY270115P00870000,2024-10-10 13:30:05+00:00,870.0,292.17,269.50,274.29,0.000000,0.000000,NaN,0,0.000010,True,REGULAR,USD,2027-01-15,put
6823,SPY270115P00900000,2024-12-16 14:51:59+00:00,900.0,294.67,306.00,310.90,0.000000,0.000000,1.0,0,0.170296,True,REGULAR,USD,2027-01-15,put


In [6]:
# Data Cleaning

def clean_data(options_df):
    # Convert expiration date to datetime
    options_df["expiration_date"] = pd.to_datetime(options_df["expiration_date"])

    # remove rows with 'null' values
    options_df = options_df[~options_df.isnull().any(axis=1)]

    # drop duplicates
    options_df = options_df.drop_duplicates(subset="contractSymbol")

    # drop columns
    options_df.drop(columns=["lastTradeDate","change","percentChange","volume","inTheMoney","contractSize","currency"],inplace=True, errors="ignore")

    # Normalise
    current_price = yf.Ticker(ticker).history(period="1d").Close.iloc[-1]
    options_df["moneyness"] = options_df["strike"] / current_price
    options_df = options_df[(options_df["moneyness"] <= 1.2) & (options_df["moneyness"] >= 0.8)]

    # time to maturity (+ filtering out long date options)
    options_df["time_to_maturity"] = (options_df.expiration_date - pd.to_datetime("today")).dt.days
    options_df = options_df[(options_df["time_to_maturity"] <= 180) & (options_df["time_to_maturity"] > 0)]

    # Filter out thinly traded options (low open interest)
    options_df = options_df[options_df.openInterest > 1000] # may need to change this -> this would need to change for diff tickers

    return options_df

options_df = clean_data(options_df)
options_df


,contractSymbol,strike,lastPrice,bid,ask,openInterest,impliedVolatility,expiration_date,type,moneyness,time_to_maturity
11,SPY250106C00580000,580.0,11.66,11.64,11.82,1220,0.000010,2025-01-06,call,0.979812,1
14,SPY250106C00583000,583.0,8.85,8.73,8.88,1389,0.000010,2025-01-06,call,0.984880,1
15,SPY250106C00585000,585.0,6.90,6.83,6.97,2435,0.061533,2025-01-06,call,0.988259,1
16,SPY250106C00586000,586.0,5.98,5.92,6.05,1650,0.071787,2025-01-06,call,0.989948,1
17,SPY250106C00587000,587.0,5.05,5.04,5.13,2093,0.071909,2025-01-06,call,0.991638,1
...,...,...,...,...,...,...,...,...,...,...,...
4267,SPY250630P00568000,568.0,15.36,13.63,13.74,1375,0.147607,2025-06-30,put,0.959540,176
4271,SPY250630P00580000,580.0,16.40,16.42,16.55,2332,0.134957,2025-06-30,put,0.979812,176
4272,SPY250630P00585000,585.0,17.86,17.79,17.93,1502,0.129586,2025-06-30,put,0.988259,176
4273,SPY250630P00590000,590.0,19.44,19.31,19.46,1195,0.124154,2025-06-30,put,0.996706,176


In [7]:
# filter calls and puts

def option_type_filter(options_df, option_type):
    return options_df[options_df['type'] == option_type]


calls_df = option_type_filter(options_df, 'call')
puts_df = option_type_filter(options_df, 'put')

In [8]:
# Outlier Detection 

def outlier_detection(df, outlier_penalty):

    # Initialize the MinMaxScaler
    scaler = StandardScaler()

    # Normalize both features
    scaled_data = scaler.fit_transform(df[['moneyness', 'time_to_maturity']])

    IV = df['impliedVolatility'].values

    # Fit a KNN model to the feature data
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(scaled_data)

    # Find the k nearest neighbors for each point
    distances, indices = knn.kneighbors(scaled_data)

    # Calculate the average IV of the neighbors
    avg_IV_neighbors = []
    for i in range(len(df)):
        # Get the IVs of the k nearest neighbors
        neighbor_IVs = IV[indices[i]]
        avg_IV_neighbors.append(np.mean(neighbor_IVs))

    # Compute the difference between the IV of each point and its neighbors' average IV
    iv_diff = np.abs(IV - avg_IV_neighbors)

    # Define a threshold for outliers 
    threshold = outlier_penalty * np.std(iv_diff)

    # Flag outliers
    outliers = iv_diff > threshold

    # Add the 'Outlier' column to the DataFrame
    df['Outlier'] = outliers

    df["Outlier_Adj_IV"] = np.where(df["Outlier"] == True, np.nan, df["impliedVolatility"])
    
    return df


outlier_detection(calls_df, outlier_penalty)
outlier_detection(puts_df, outlier_penalty)

C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_53636\3112570997.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Outlier'] = outliers
C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_53636\3112570997.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outlier_Adj_IV"] = np.where(df["Outlier"] == True, np.nan, df["impliedVolatility"])
C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_53636\3112570997.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,contractSymbol,strike,lastPrice,bid,ask,openInterest,impliedVolatility,expiration_date,type,moneyness,time_to_maturity,Outlier,Outlier_Adj_IV
69,SPY250106P00485000,485.0,0.01,0.00,0.01,1164,0.671878,2025-01-06,put,0.819326,1,True,NaN
81,SPY250106P00560000,560.0,0.01,0.01,0.02,1267,0.238289,2025-01-06,put,0.946026,1,True,NaN
86,SPY250106P00565000,565.0,0.01,0.01,0.02,1218,0.203133,2025-01-06,put,0.954472,1,True,NaN
88,SPY250106P00567000,567.0,0.01,0.01,0.02,1704,0.189461,2025-01-06,put,0.957851,1,False,0.189461
89,SPY250106P00568000,568.0,0.02,0.02,0.03,1275,0.191414,2025-01-06,put,0.959540,1,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,SPY250630P00568000,568.0,15.36,13.63,13.74,1375,0.147607,2025-06-30,put,0.959540,176,False,0.147607
4271,SPY250630P00580000,580.0,16.40,16.42,16.55,2332,0.134957,2025-06-30,put,0.979812,176,False,0.134957
4272,SPY250630P00585000,585.0,17.86,17.79,17.93,1502,0.129586,2025-06-30,put,0.988259,176,False,0.129586
4273,SPY250630P00590000,590.0,19.44,19.31,19.46,1195,0.124154,2025-06-30,put,0.996706,176,False,0.124154


In [9]:
# Plot for calls
fig_calls = px.scatter_3d(
    calls_df,
    x="moneyness", 
    y="time_to_maturity",   
    z="impliedVolatility",  
    color="Outlier",
    hover_data=["contractSymbol","Outlier"],
    title="Call Options - Implied Volatility Surface",
)

fig_calls.update_traces(marker=dict(size=3))  # Adjust marker size
fig_calls.show()


# # Plot for puts
fig_puts = px.scatter_3d(
    puts_df,
    x="moneyness",  # x-axis (strike price)
    y="time_to_maturity",   # y-axis (time to maturity)
    z="impliedVolatility",  # z-axis (IV)
    color="Outlier",  # Optional: Color by expiration date
    hover_data="contractSymbol",
    title="Put Options - Implied Volatility Surface"
)
fig_puts.update_traces(marker=dict(size=3))
fig_puts.show()



In [10]:
print(calls_df.head())



        contractSymbol  strike  lastPrice    bid    ask  openInterest  \
11  SPY250106C00580000   580.0      11.66  11.64  11.82          1220   
14  SPY250106C00583000   583.0       8.85   8.73   8.88          1389   
15  SPY250106C00585000   585.0       6.90   6.83   6.97          2435   
16  SPY250106C00586000   586.0       5.98   5.92   6.05          1650   
17  SPY250106C00587000   587.0       5.05   5.04   5.13          2093   

    impliedVolatility expiration_date  type  moneyness  time_to_maturity  \
11           0.000010      2025-01-06  call   0.979812                 1   
14           0.000010      2025-01-06  call   0.984880                 1   
15           0.061533      2025-01-06  call   0.988259                 1   
16           0.071787      2025-01-06  call   0.989948                 1   
17           0.071909      2025-01-06  call   0.991638                 1   

    Outlier  Outlier_Adj_IV  
11     True             NaN  
14     True             NaN  
15     True   

In [31]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

def cubic_interpolation(df):
    strike_grid = np.linspace(df['moneyness'].min(), df['moneyness'].max(), 50)
    maturity_grid = np.linspace(df['time_to_maturity'].min(), df['time_to_maturity'].max(), 50)

    # This creates a meshgrid of strike price and time to maturities
    X, Y = np.meshgrid(strike_grid, maturity_grid)

    # Perform cubic spline interpolation to get implied volatilities for the grid points
    Z = griddata(
        (df['moneyness'], df['time_to_maturity']),
        df['impliedVolatility'],                  # Outlier_Adj_IV
        (X, Y),
        method='cubic'
    )

    # Z_smoothed = gaussian_filter(Z, sigma=1)

    # Create the 3D surface plot using plotly.graph_objects
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

    # Add title and labels
    fig.update_layout(
        title="Implied Volatility Surface (Cubic Spline)",
        scene=dict(
            xaxis_title='Strike Price',
            yaxis_title='Time to Maturity',
            zaxis_title='Implied Volatility'
        )
    )

    # Show the plot
    fig.show()


cubic_interpolation(calls_df)
cubic_interpolation(puts_df)



In [29]:
print(fig)

Figure({
    'data': [{'type': 'surface',
              'x': array([[0.80243262, 0.90168087, 1.00092911, 1.10017736, 1.1994256 ],
                          [0.80243262, 0.90168087, 1.00092911, 1.10017736, 1.1994256 ],
                          [0.80243262, 0.90168087, 1.00092911, 1.10017736, 1.1994256 ],
                          [0.80243262, 0.90168087, 1.00092911, 1.10017736, 1.1994256 ],
                          [0.80243262, 0.90168087, 1.00092911, 1.10017736, 1.1994256 ]]),
              'y': array([[  1.  ,   1.  ,   1.  ,   1.  ,   1.  ],
                          [ 44.75,  44.75,  44.75,  44.75,  44.75],
                          [ 88.5 ,  88.5 ,  88.5 ,  88.5 ,  88.5 ],
                          [132.25, 132.25, 132.25, 132.25, 132.25],
                          [176.  , 176.  , 176.  , 176.  , 176.  ]]),
              'z': array([[        nan,         nan,  0.06894924,         nan,         nan],
                          [ 0.99407829,  0.8540842 ,  0.17117528,  0.11474993,  0

In [15]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter

# Scatter Plot for Calls (market data)
fig = go.Figure()

# Plot calls market data
fig.add_trace(go.Scatter3d(
    x=calls_df['moneyness'],
    y=calls_df['time_to_maturity'],
    z=calls_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=calls_df['impliedVolatility'], colorscale='Viridis'),
    name="Calls Market IV"
))

# Plot puts market data
fig.add_trace(go.Scatter3d(
    x=puts_df['moneyness'],
    y=puts_df['time_to_maturity'],
    z=puts_df['impliedVolatility'],
    mode='markers',
    marker=dict(size=3, color=puts_df['impliedVolatility'], colorscale='Cividis'),
    name="Puts Market IV"
))

# Create a grid for interpolation (for the surface plot)
strike_grid = np.linspace(calls_df['moneyness'].min(), calls_df['moneyness'].max(), 50)
maturity_grid = np.linspace(calls_df['time_to_maturity'].min(), calls_df['time_to_maturity'].max(), 50)

# Create a meshgrid of strike price and time to maturity
X, Y = np.meshgrid(strike_grid, maturity_grid)

# Perform cubic spline interpolation to get implied volatilities for the grid points
Z = griddata(
    (calls_df['moneyness'], calls_df['time_to_maturity']),
    calls_df['impliedVolatility'],
    (X, Y),
    method='cubic'
)

# Optionally smooth the Z values with a Gaussian filter
Z_smoothed = gaussian_filter(Z, sigma=1.5)

# Add the smoothed implied volatility surface
fig.add_trace(go.Surface(
    z=Z_smoothed,
    x=X,
    y=Y,
    colorscale='Viridis',
    opacity=0.5,
    name="Smoothed IV Surface"
))

# Add title and labels
fig.update_layout(
    title="Implied Volatility Surface (Calls & Puts) with Interpolated Surface",
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Time to Maturity',
        zaxis_title='Implied Volatility'
    )
)

# Show the combined plot
fig.show()


In [16]:
####### THE CODE BELOW IS TEST ONLY

# This looks at slicing the Vol surface by time and moneyness

In [17]:
import plotly.graph_objects as go

# Choose a fixed maturity (time to maturity)
fixed_maturity = 180  # Example: choose 30 days to maturity
# Find the index closest to the fixed maturity
maturity_index = np.abs(maturity_grid - fixed_maturity).argmin()

# Extract the corresponding IV values for the fixed maturity
iv_slice = Z_smoothed[maturity_index, :]

# Extract the strike prices (x-axis) for the slice
strike_slice = X[maturity_index, :]

# Market data for calls at the fixed maturity (you might filter your dataframe accordingly)
calls_at_maturity = calls_df[calls_df['time_to_maturity'] == fixed_maturity]

# Create the 2D plot for the slice
fig_slice_maturity = go.Figure()

# Add a line plot showing the slice for the fixed maturity
fig_slice_maturity.add_trace(go.Scatter(
    x=strike_slice,
    y=iv_slice,
    mode='lines',
    name=f"IV Slice at Maturity = {fixed_maturity} days",
    line=dict(color='blue')
))

# Overlay the scatter points of the market data
fig_slice_maturity.add_trace(go.Scatter(
    x=calls_at_maturity['moneyness'],
    y=calls_at_maturity['impliedVolatility'],
    mode='markers',
    name="Market Data",
    marker=dict(color='red', size=5)
))

# Add titles and labels
fig_slice_maturity.update_layout(
    title=f"Implied Volatility Slice at Maturity = {fixed_maturity} days",
    xaxis_title="Strike Price (Normalized)",
    yaxis_title="Implied Volatility"
)

# Show the plot
fig_slice_maturity.show()



In [18]:
# Choose a fixed strike price (moneyness)
fixed_strike = 1.0  # Example: choose normalized strike price of 1 (ATM)
# Find the index closest to the fixed strike
strike_index = np.abs(strike_grid - fixed_strike).argmin()

# Extract the corresponding IV values for the fixed strike
iv_slice_strike = Z_smoothed[:, strike_index]

# Extract the maturities (y-axis) for the slice
maturity_slice = Y[:, strike_index]

# Market data for calls at the fixed strike (filter your dataframe accordingly)
calls_at_strike = calls_df[calls_df['moneyness'] == fixed_strike]

# Create the 2D plot for the slice
fig_slice_strike = go.Figure()

# Add a line plot showing the slice for the fixed strike
fig_slice_strike.add_trace(go.Scatter(
    x=maturity_slice,
    y=iv_slice_strike,
    mode='lines',
    name=f"IV Slice at Strike = {fixed_strike}",
    line=dict(color='blue')
))

# Overlay the scatter points of the market data
fig_slice_strike.add_trace(go.Scatter(
    x=calls_at_strike['time_to_maturity'],
    y=calls_at_strike['impliedVolatility'],
    mode='markers',
    name="Market Data",
    marker=dict(color='red', size=5)
))

# Add titles and labels
fig_slice_strike.update_layout(
    title=f"Implied Volatility Slice at Strike = {fixed_strike}",
    xaxis_title="Time to Maturity (Days)",
    yaxis_title="Implied Volatility"
)

# Show the plot
fig_slice_strike.show()

